In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [2]:
rs=42

In [3]:
import os
print(os.getcwd())
os.chdir('/home/CNIO.ES/asanchezb/esophageous_cancer/intensities/')
print(os.getcwd())

/home/CNIO.ES/asanchezb
/home/CNIO.ES/asanchezb/esophageous_cancer/intensities


In [4]:
# load data
probe_df = pd.read_csv("probe_df.csv", index_col = 0)

In [31]:
probe_df.head(5)

,1,2,3,4,5,6,7,8,9,10,...,1018,1019,1020,1022,1023,1024,1025,1026,1027,new_y
A49O,3135.625296,8760.851652,5360.432341,5172.712077,5233.644668,2542.196190,13336.316282,9677.131349,15050.814755,17708.192698,...,8007.021207,10499.697372,16014.524813,15881.666719,7199.586570,8199.400075,4731.231335,8479.416133,10944.625645,1
A4OE,4029.770264,9957.709799,5348.619569,4878.058197,5781.628790,1995.797161,15250.010204,10889.043799,14424.648933,18464.339992,...,8738.164774,12740.142710,12806.549471,17224.717583,7301.979864,8354.332888,4427.331635,11030.730326,12699.300458,1
A49Y,3629.215222,7836.251094,10034.753534,5721.899299,12767.090194,2098.048596,18443.726258,14569.545080,9148.577106,18135.189468,...,7784.165600,13679.293085,11280.683719,16078.832146,7742.162199,7453.207026,910.495871,6764.409866,7674.434722,1
A4OP,3864.993238,9349.041369,5012.657863,4110.092220,5086.318105,1700.948527,14093.355120,7745.291077,13438.458253,19935.798687,...,8987.334560,9694.027202,9334.539636,17825.721079,7608.428782,7252.553632,3181.335974,10167.867452,9988.848056,1
A56G,3074.681315,9126.187312,6113.792356,5292.708353,3196.921837,1865.339971,22551.507507,9301.671464,18225.746730,23134.759092,...,8957.739634,14370.058116,12203.922426,22035.096350,8101.613103,7999.452875,3841.185892,12541.697507,13070.474198,1


In [46]:
# x and y split
x = probe_df.iloc[:,:-1]
y = probe_df.iloc[:,-1]

In [14]:
# Normalize data
scaler = MinMaxScaler()
scaler.fit_transform(x)

array([[0.31893136, 0.35292881, 0.21955277, ..., 0.53188055, 0.29183638,
        0.39232158],
       [0.48592234, 0.44352963, 0.2184778 , ..., 0.48957499, 0.55590236,
        0.54656708],
       [0.41111447, 0.28293758, 0.64492084, ..., 0.        , 0.1143299 ,
        0.10485394],
       ...,
       [0.22399993, 0.6486433 , 0.15135082, ..., 0.23598801, 0.43681503,
        0.2003932 ],
       [0.44021989, 0.44957011, 0.28315427, ..., 0.35213922, 0.28855615,
        0.36204809],
       [0.01593375, 0.82594589, 0.0880131 , ..., 0.68314699, 0.38189326,
        0.27099792]])

In [49]:
cv_outer = KFold(n_splits=5, shuffle=True, random_state=rs)

In [47]:
x

,1,2,3,4,5,6,7,8,9,10,...,1017,1018,1019,1020,1022,1023,1024,1025,1026,1027
A49O,3135.625296,8760.851652,5360.432341,5172.712077,5233.644668,2542.196190,13336.316282,9677.131349,15050.814755,17708.192698,...,1486.157313,8007.021207,10499.697372,16014.524813,15881.666719,7199.586570,8199.400075,4731.231335,8479.416133,10944.625645
A4OE,4029.770264,9957.709799,5348.619569,4878.058197,5781.628790,1995.797161,15250.010204,10889.043799,14424.648933,18464.339992,...,3323.962667,8738.164774,12740.142710,12806.549471,17224.717583,7301.979864,8354.332888,4427.331635,11030.730326,12699.300458
A49Y,3629.215222,7836.251094,10034.753534,5721.899299,12767.090194,2098.048596,18443.726258,14569.545080,9148.577106,18135.189468,...,885.834879,7784.165600,13679.293085,11280.683719,16078.832146,7742.162199,7453.207026,910.495871,6764.409866,7674.434722
A4OP,3864.993238,9349.041369,5012.657863,4110.092220,5086.318105,1700.948527,14093.355120,7745.291077,13438.458253,19935.798687,...,2725.173582,8987.334560,9694.027202,9334.539636,17825.721079,7608.428782,7252.553632,3181.335974,10167.867452,9988.848056
A56G,3074.681315,9126.187312,6113.792356,5292.708353,3196.921837,1865.339971,22551.507507,9301.671464,18225.746730,23134.759092,...,2704.192376,8957.739634,14370.058116,12203.922426,22035.096350,8101.613103,7999.452875,3841.185892,12541.697507,13070.474198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A49V,2298.507208,9190.715509,5162.295007,4918.146100,5895.519038,2278.904369,9569.529647,10155.409338,17115.077839,16557.865051,...,3365.892285,6752.343418,9378.443594,7663.620534,13867.985519,9010.659629,6859.473563,3433.481780,5884.023946,9476.845610
A49L,3987.996508,8414.987221,5251.033483,5945.422517,6064.708895,1985.777150,17624.952714,9722.528685,14711.537179,12410.016177,...,3287.981058,6331.160583,11238.441993,11119.798370,14502.251871,10171.518173,7782.683905,4052.490804,7281.319685,9845.455695
A8NR,2627.319659,12667.309697,4610.968792,3074.824991,7866.749039,2552.916429,17465.513446,10458.707206,16245.573622,25977.428359,...,1553.757478,8499.841448,11121.680293,15239.037190,24222.600894,11032.677126,8539.084417,2605.703098,9880.149750,8761.275758
A4OR,3785.058737,10037.505948,6059.341955,5083.977239,5430.924112,1910.025753,12519.648650,11501.842069,12462.016273,22230.194564,...,1460.819212,9644.895571,12582.831876,10789.672946,19568.730413,7472.654987,7599.990506,3440.069136,8447.723695,10600.238770


In [58]:
outer_results = []

In [61]:
for train_ix, test_ix in cv_outer.split(x):
    
    x_train, x_test = x.iloc[train_ix,:], x.iloc[test_ix,:]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    cv_inner = KFold(n_splits=3, shuffle=True, random_state=rs)
    model=svm.SVC(kernel='rbf')
    par = {'C':[1,10,100], 'gamma':['scale',0.001,0.01,0.1]}
    search = GridSearchCV(model,par,scoring='accuracy',cv=cv_inner, refit=True)
    result = search.fit(x_train,y_train)
    best_model = result.best_estimator_
    yhat = best_model.predict(x_test)
    acc = metrics.accuracy_score(y_test,yhat)
    outer_results.append(acc)
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))

print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.769, est=0.759, cfg={'C': 1, 'gamma': 0.01}
>acc=0.731, est=0.769, cfg={'C': 1, 'gamma': 0.01}
>acc=0.769, est=0.760, cfg={'C': 1, 'gamma': 0.01}
>acc=0.731, est=0.769, cfg={'C': 1, 'gamma': 0.01}
>acc=0.808, est=0.750, cfg={'C': 1, 'gamma': 0.01}
Accuracy: 0.763 (0.026)


In [75]:
print(result.)

3
